In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'f9bmulqb'

In [3]:
wandb.init(id='f9bmulqb', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13332.809

In [5]:
m.backbone

EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
 

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [8]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.48 µs


In [10]:
#wandb.watch(m, log="all")

In [11]:
model_stem = "7.1"
START_E = 0

In [12]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 3e-4 if e<6 else 1e-4
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.00398725, 'consistency losses/trn_steer_cost': 0.0040776, 'consistency losses/trn_te_loss': 3.22677443, 'consistency losses/trn_torque_loss': 384.05807087, 'consistency losses/trn_torque_delta_loss': 109.91113281, 'logistical/obs_consumed_per_second': 236.51181102, 'logistical/obs_generated_per_second': 110.15748031, 'logistical/data_consumption_ratio': 2.15318794, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.24176216, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 233.9375, 'logistical/obs_generated_per_second': 109.8671875, 'logistical/data_consumption_ratio': 2.12727504, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00382146, 'consistency losses/trn_steer_cost': 0.00401963, 'consistency losses/trn_te_loss': 3.22885823, 'consistency losses/trn_torque_loss': 374.65802846, 'consistency losses/trn_torque_delta_loss': 101.171875, 'logistical/max_param': 16.24435043, 'logistical/lr': 0.0003}
{'logistical/ob

{'logistical/obs_consumed_per_second': 232.2578125, 'logistical/obs_generated_per_second': 100.5703125, 'logistical/data_consumption_ratio': 2.31635548, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00355349, 'consistency losses/trn_steer_cost': 0.00311857, 'consistency losses/trn_te_loss': 3.22629479, 'consistency losses/trn_torque_delta_loss': 104.30932617, 'consistency losses/trn_torque_loss': 375.91340042, 'logistical/max_param': 16.24913788, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 239.0, 'logistical/obs_generated_per_second': 108.6640625, 'logistical/data_consumption_ratio': 2.20970921, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00385246, 'consistency losses/trn_steer_cost': 0.00281297, 'consistency losses/trn_te_loss': 3.19951581, 'consistency losses/trn_torque_loss': 350.84572072, 'consistency losses/trn_torque_delta_loss': 96.00390625, 'logistical/max_param': 16.2538681, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_pe

{'logistical/obs_consumed_per_second': 239.5625, 'logistical/obs_generated_per_second': 97.2265625, 'logistical/data_consumption_ratio': 2.46578013, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00305764, 'consistency losses/trn_steer_cost': 0.00320216, 'consistency losses/trn_te_loss': 3.22319087, 'consistency losses/trn_torque_loss': 374.80133929, 'consistency losses/trn_torque_delta_loss': 97.76611328, 'logistical/max_param': 16.2440033, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 236.859375, 'logistical/obs_generated_per_second': 102.8359375, 'logistical/data_consumption_ratio': 2.30529209, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00343917, 'consistency losses/trn_steer_cost': 0.0032663, 'consistency losses/trn_te_loss': 3.2686209, 'consistency losses/trn_torque_loss': 362.24631606, 'consistency losses/trn_torque_delta_loss': 100.68920898, 'logistical/max_param': 16.24422646, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per

{'logistical/obs_consumed_per_second': 245.3359375, 'logistical/obs_generated_per_second': 108.3203125, 'logistical/data_consumption_ratio': 2.26169207, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00345286, 'consistency losses/trn_steer_cost': 0.00328883, 'consistency losses/trn_te_loss': 3.12055984, 'consistency losses/trn_torque_delta_loss': 99.23583984, 'consistency losses/trn_torque_loss': 406.68948864, 'logistical/max_param': 16.23885155, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 237.15625, 'logistical/obs_generated_per_second': 106.7109375, 'logistical/data_consumption_ratio': 2.22421221, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00317131, 'consistency losses/trn_steer_cost': 0.00313504, 'consistency losses/trn_te_loss': 3.24624897, 'consistency losses/trn_torque_loss': 359.32913306, 'consistency losses/trn_torque_delta_loss': 98.3112793, 'logistical/max_param': 16.23925591, 'logistical/lr': 0.0003}
{'logistical/obs_consumed

{'logistical/obs_consumed_per_second': 235.78125, 'logistical/obs_generated_per_second': 96.5703125, 'logistical/data_consumption_ratio': 2.44393815, 'logistical/manual_train_pause': 0.00023438, 'trn_control_loss': 0.00283998, 'consistency losses/trn_steer_cost': 0.00324906, 'consistency losses/trn_te_loss': 3.33182959, 'consistency losses/trn_torque_loss': 360.20181197, 'consistency losses/trn_torque_delta_loss': 101.03686523, 'logistical/max_param': 16.24008942, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.359375, 'logistical/obs_generated_per_second': 90.0703125, 'logistical/data_consumption_ratio': 2.61536845, 'logistical/manual_train_pause': 0.00273438, 'trn_control_loss': 0.00296038, 'consistency losses/trn_steer_cost': 0.00358009, 'consistency losses/trn_te_loss': 3.25845841, 'consistency losses/trn_torque_loss': 463.20507812, 'consistency losses/trn_torque_delta_loss': 105.61767578, 'logistical/max_param': 16.24082184, 'logistical/lr': 0.0001}
{'logistica

...

{'logistical/obs_consumed_per_second': 242.9609375, 'logistical/obs_generated_per_second': 109.4296875, 'logistical/data_consumption_ratio': 2.22660689, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00262044, 'consistency losses/trn_steer_cost': 0.00299928, 'consistency losses/trn_te_loss': 3.25365717, 'consistency losses/trn_torque_loss': 371.56070243, 'consistency losses/trn_torque_delta_loss': 96.85571289, 'logistical/max_param': 16.23687363, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.8359375, 'logistical/obs_generated_per_second': 116.8515625, 'logistical/data_consumption_ratio': 2.06133559, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00279201, 'consistency losses/trn_steer_cost': 0.00339925, 'consistency losses/trn_te_loss': 3.20098469, 'consistency losses/trn_torque_loss': 336.50854701, 'consistency losses/trn_torque_delta_loss': 96.88256836, 'logistical/max_param': 16.23697472, 'logistical/lr': 0.0001}
{'logistical/obs_consu

{'logistical/obs_consumed_per_second': 233.6015625, 'logistical/obs_generated_per_second': 102.7578125, 'logistical/data_consumption_ratio': 2.27865788, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00246505, 'consistency losses/trn_steer_cost': 0.00312013, 'consistency losses/trn_te_loss': 3.27275976, 'consistency losses/trn_torque_loss': 300.97826087, 'consistency losses/trn_torque_delta_loss': 94.48461914, 'logistical/max_param': 16.2377739, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.375, 'logistical/obs_generated_per_second': 102.15625, 'logistical/data_consumption_ratio': 2.36039278, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00261639, 'consistency losses/trn_steer_cost': 0.00317203, 'consistency losses/trn_te_loss': 3.17012625, 'consistency losses/trn_torque_loss': 383.74854343, 'consistency losses/trn_torque_delta_loss': 96.1105957, 'logistical/max_param': 16.24005318, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_

{'logistical/obs_consumed_per_second': 230.0078125, 'logistical/obs_generated_per_second': 107.8984375, 'logistical/data_consumption_ratio': 2.13469907, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00257433, 'consistency losses/trn_steer_cost': 0.00363747, 'consistency losses/trn_te_loss': 3.39305438, 'consistency losses/trn_torque_loss': 374.19364754, 'consistency losses/trn_torque_delta_loss': 97.27441406, 'logistical/max_param': 16.24015236, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 236.3671875, 'logistical/obs_generated_per_second': 100.9921875, 'logistical/data_consumption_ratio': 2.35002607, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00200156, 'consistency losses/trn_steer_cost': 0.00243181, 'consistency losses/trn_te_loss': 3.23937625, 'consistency losses/trn_torque_loss': 226.86921875, 'consistency losses/trn_torque_delta_loss': 90.30883789, 'logistical/max_param': 16.24056053, 'logistical/lr': 0.0001}
{'logistical/obs_consu

{'logistical/obs_consumed_per_second': 238.59375, 'logistical/obs_generated_per_second': 105.6015625, 'logistical/data_consumption_ratio': 2.26272433, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00267021, 'consistency losses/trn_steer_cost': 0.00303266, 'consistency losses/trn_te_loss': 3.16230151, 'consistency losses/trn_torque_loss': 312.06559806, 'consistency losses/trn_torque_delta_loss': 92.99389648, 'logistical/max_param': 16.24198151, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.5625, 'logistical/obs_generated_per_second': 114.3203125, 'logistical/data_consumption_ratio': 2.10525682, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0021495, 'consistency losses/trn_steer_cost': 0.00257697, 'consistency losses/trn_te_loss': 3.25977197, 'consistency losses/trn_torque_loss': 315.80568484, 'consistency losses/trn_torque_delta_loss': 86.02758789, 'logistical/max_param': 16.24206543, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_pe

KeyboardInterrupt: 

In [20]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e29.torch")

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
